In [1]:
# KAGGLE ---------------------
import sys
import numpy as np
import keras
from keras.utils import Sequence
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf


/Users/farrar/py3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# MINE -----------------------------------
import hprotein

In [3]:
# KAGGLE ---------------------------------
BATCH_SIZE = 2
SEED = 777
SHAPE = (512, 512, 4)
DIR = '../input'
VAL_RATIO = 0.1 # 10 % as validation
THRESHOLD = 0.5 # due to different cost of True Positive vs False Positive, this is the probability threshold to predict the class as 'yes'

ia.seed(SEED)

In [4]:
# MINE -----------------------------------
# constants
TRAIN_PATH = '../stage1_train'
TEST_PATH = '../stage1_test'
LABEL_PATH = '../stage1_labels/train_short.csv'
OUTPUT_PATH = '../stage1_submit'
COLORS = ['red','green', 'blue', 'yellow']
IMAGE_SIZE = 512
CROP_SIZE = 512
BATCH_SIZE = 2
SHAPE = (CROP_SIZE, CROP_SIZE, 4)
THRESHOLD = 0.05
SEED = 42


In [5]:
# KAGGLE ---------------------
def getTrainDataset():
    
    # path_to_train = DIR + '/train/'
    # data = pd.read_csv(DIR + '/train.csv')

    path_to_train = TRAIN_PATH
    data = pd.read_csv(LABEL_PATH)
    
    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

# unit test ------------------------------------
p, l = getTrainDataset()
print (len(p))
print (p)
print (len(l))
print(type(l[0][1]))
print((l[0][1]))
print (l)

10
['../stage1_train/000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 '../stage1_train/001838f8-bbca-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/fea6e496-bbbb-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/ffeae6f0-bbc9-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0']
10
<class 'numpy.float64'>
1.0
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [14]:
# MINE -----------------------------------
p_me, l_me = hprotein.get_data(TRAIN_PATH, LABEL_PATH, mode='train', filter_ids=hprotein.mini_validate_set)
print (len(p_me))
print (p_me)
print (len(l_me))
print(type(l_me[0][1]))
print((l_me[0][1]))
print (l_me)

7
['001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 'fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0'
 '0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0'
 'fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 'fea6e496-bbbb-11e8-b2ba-ac1f6b6435d0']
7
<class 'numpy.float64'>
0.0
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [8]:
# KAGGLE ---------------------
def getTestDataset():
    
    #path_to_test = DIR + '/test/'
    #data = pd.read_csv(DIR + '/sample_submission.csv')
    path_to_test = TRAIN_PATH
    data = pd.read_csv(OUTPUT_PATH + '/sample_submission_short.csv')


    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

# unit test ------------------------------------
p_test, l_test = getTestDataset()
print (len(p_test))
print (p_test)
print (len(l_test))
print (l_test)

7
['../stage1_train/00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0'
 '../stage1_train/00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0'
 '../stage1_train/000cce7e-bad4-11e8-b2b8-ac1f6b6435d0'
 '../stage1_train/0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'
 '../stage1_train/0008baca-bad7-11e8-b2b9-ac1f6b6435d0'
 '../stage1_train/003170fa-bacd-11e8-b2b8-ac1f6b6435d0'
 '../stage1_train/0031820a-baca-11e8-b2b8-ac1f6b6435d0']
7
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [16]:
# MINE -----------------------------------
p_me_test, l_me_test = hprotein.get_data(TEST_PATH, OUTPUT_PATH, mode='test', filter_ids=None)
print (len(p_me_test))
print (p_me_test)
print (len(l_me_test))
print (l_me_test)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# KAGGLE -----------------------------------
class ProteinDataGenerator(keras.utils.Sequence):
            
    def __init__(self, paths, labels, batch_size, shape, shuffle = False, use_cache = False, augment = False):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.use_cache = use_cache
        self.augment = augment
        if use_cache == True:
            self.cache = np.zeros((paths.shape[0], shape[0], shape[1], shape[2]), dtype=np.float16)
            self.is_cached = np.zeros((paths.shape[0]))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
        # Generate data
        if self.use_cache == True:
            X = self.cache[indexes]
            for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
                image = self.__load_image(path)
                self.is_cached[indexes[i]] = 1
                self.cache[indexes[i]] = image
                X[i] = image
        else:
            for i, path in enumerate(paths):
                X[i] = self.__load_image(path)

        y = self.labels[indexes]
        
        if self.augment == True:
            seq = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Strengthen or weaken the contrast in each image.
                    iaa.ContrastNormalization((0.75, 1.5)),
                    # Add gaussian noise.
                    # For 50% of all images, we sample the noise once per pixel.
                    # For the other 50% of all images, we sample the noise per pixel AND
                    # channel. This can change the color (not only brightness) of the
                    # pixels.
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])], random_order=True)
        
            X = np.concatenate((X, seq.augment_images(X), seq.augment_images(X), seq.augment_images(X)), 0)
            y = np.concatenate((y, y, y, y), 0)
        
        return X, y
    
    def on_epoch_end(self):
        
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        R = Image.open(path + '_red.png')
        G = Image.open(path + '_green.png')
        B = Image.open(path + '_blue.png')
        Y = Image.open(path + '_yellow.png')

        im = np.stack((
            np.array(R), 
            np.array(G), 
            np.array(B),
            np.array(Y)), -1)

        #print('Before resize -> {}'.format(im.shape))
        im = cv2.resize(im, (SHAPE[0], SHAPE[1]))
        #print('After resize -> {}'.format(im.shape))
        im = np.divide(im, 255)
        return im

In [ ]:
# KAGGLE -----------------------------------
paths, labels = getTrainDataset()

# divide to 
keys = np.arange(paths.shape[0], dtype=np.int)  
np.random.seed(SEED)
np.random.shuffle(keys)
lastTrainIndex = int((1-VAL_RATIO) * paths.shape[0])

pathsTrain = paths[0:lastTrainIndex]
labelsTrain = labels[0:lastTrainIndex]
pathsVal = paths[lastTrainIndex:]
labelsVal = labels[lastTrainIndex:]

print(paths.shape, labels.shape)
print(pathsTrain.shape, labelsTrain.shape, pathsVal.shape, labelsVal.shape)
print(pathsTrain)
print(labelsTrain)
print(pathsVal)
print(labelsVal)

In [ ]:
# MINE -----------------------------------
class myargs:
    def __init__(self):
        self.train_folder = TRAIN_PATH
        self.label_folder = LABEL_PATH
        self.batch_size = BATCH_SIZE
        self.validation_steps = 8
        self.epochs = 5
    
args = myargs()

train_set_sids, val_set_sids, train_set_lbls, val_set_lbls = hprotein.get_train_test_split(args, test_size=1)
print(train_set_sids.shape, train_set_lbls.shape, val_set_sids.shape, val_set_lbls.shape)
print(train_set_sids)
print(train_set_lbls)
print(val_set_sids)
print(val_set_lbls)

In [ ]:
# KAGGLE -----------------------------------
tg = ProteinDataGenerator(pathsTrain, labelsTrain, BATCH_SIZE, SHAPE, use_cache=False, augment = True, shuffle = False)
vg = ProteinDataGenerator(pathsVal, labelsVal, BATCH_SIZE, SHAPE, use_cache=False, shuffle = False)
print(tg[0])

In [ ]:
# MINE -----------------------------------
train_specimen_ids = train_set_sids
train_labels = train_set_lbls
val_specimen_ids = val_set_sids
val_labels = val_set_lbls

training_generator = hprotein.HproteinDataGenerator(args, args.train_folder, train_specimen_ids, train_labels, shuffle=True)
val_generator = hprotein.HproteinDataGenerator(args, args.train_folder, val_specimen_ids, val_labels)
print(training_generator[0][0].shape)

In [ ]:
# KAGGLE -----------------------------------
def f1(y_true, y_pred):
    #y_pred = K.round(y_pred)
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))

In [ ]:
# MINE -----------------------------------
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

my_ftest = hprotein.f1(y_true, y_pred)
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(my_ftest))

In [ ]:
# KAGGLE -----------------------------------
def f1_loss(y_true, y_pred):
    
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1_loss(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))

In [ ]:
# MINE -----------------------------------
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

my_ftest = hprotein.f1_loss(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(my_ftest))

In [ ]:
# KAGGLE -----------------------------------
def create_model(input_shape):
    
    dropRate = 0.25
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(32, (3, 3))(x) #, strides=(2,2))(x)
    x = ReLU()(x)

    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp1 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp1)
    x = Conv2D(64, (3, 3), strides=(2,2))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp2 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp2)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    ginp3 = Dropout(dropRate)(x)
    
    gap1 = GlobalAveragePooling2D()(ginp1)
    gap2 = GlobalAveragePooling2D()(ginp2)
    gap3 = GlobalAveragePooling2D()(ginp3)
    
    x = Concatenate()([gap1, gap2, gap3])
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

# unit
model = create_model(SHAPE)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-03),
    metrics=['acc',hprotein.f1])

model.summary()

In [ ]:
# MINE -----------------------------------
my_model = hprotein.create_model(hprotein.SHAPE, model_name='gap_net_selu')
my_model.compile(loss='binary_crossentropy', optimizer=Adam(1e-03), metrics=['acc', hprotein.f1])
my_model.summary()

In [ ]:
# KAGGLE -----------------------------------
checkpoint = ModelCheckpoint('./base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')

In [ ]:
# MINE -----------------------------------
my_checkpoint = ModelCheckpoint('{}.model'.format('model_12345'),
                                 monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min', period=1)
reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, verbose=1, mode='min')

In [ ]:
# KAGGLE -----------------------------------
epochs = 5

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])

In [ ]:
# MINE -----------------------------------
my_hist = my_model.fit_generator(training_generator,
                               steps_per_epoch=len(training_generator),
                               validation_data=val_generator,
                               validation_steps=args.validation_steps,
                               epochs=args.epochs,
                               use_multiprocessing=False,
#                               max_queue_size=128,
                               workers=1,
                               verbose=1,
                               callbacks=[my_checkpoint,reduce_learning_rate])

In [ ]:
# MY DATA GENERATORS WITH KAGGLE MODEL -----------------------------------
checkpoint = ModelCheckpoint('./my_base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')

epochs = 5

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator),
    validation_data=val_generator,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])